In [3]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
# import torch

model_name = "openai/clip-vit-base-patch32"  
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
import os
import torch
gallery_path='gallery/'
image_extensions=['.jpg','.jpeg','.png']
image_paths=[]
for filename in os.listdir(gallery_path):
    if any(filename.lower().endswith(ext) for ext in image_extensions):
        image_paths.append(os.path.join(gallery_path,filename))

In [5]:
import os
import torch
gallery_path='gallery/'
image_extensions=['.jpg','.jpeg','.png']
image_paths=[]
for filename in os.listdir(gallery_path):
    if any(filename.lower().endswith(ext) for ext in image_extensions):
        image_paths.append(os.path.join(gallery_path,filename))

In [6]:
import numpy as np
import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# from tensorflow import device
all_embeddings=[]
all_filenames=[]
with torch.no_grad():
    for img_path in image_paths:
        try:
            image=Image.open(img_path)
            inputs=processor(images=image,return_tensors='pt')
            inputs = {k: v.to(device) for k, v in inputs.items()}
            image_embedding=model.get_image_features(**inputs)
            all_filenames.append(img_path)
            
            
           
            
            
           
            all_embeddings.append(image_embedding)
            
        except Exception as e:
            print(f'Error processing {img_path}:{e}')
if all_embeddings:
    all_embeddings_tensor=torch.stack(all_embeddings)
    all_embeddings_tensor=all_embeddings_tensor.cpu()
else:
    all_embeddings_tensor=torch.empty(0)
index_data={
    'embeddings':all_embeddings_tensor,
    'filenames':all_filenames
}

with open('index.pkl','wb') as f:
    pickle.dump(index_data,f)           


In [7]:
try:
    with open('index.pkl','rb') as f:
        index_data=pickle.load(f)
except FileNotFoundError:
    print(f'index file not found')
gallery_embeddings=index_data['embeddings'].to(device)
gallery_filenames=index_data['filenames']

In [8]:
import torch.nn.functional as F
while True:
    search_query=input('Enter a search query (or type \'q\' to quit):')
    if search_query.lower()=='q':
        print('Exiting...')
        break
    if not search_query:
        continue
    with torch.no_grad():
        inputs=processor(text=search_query,return_tensors='pt').to(device)
        text_embedding=model.get_text_features(**inputs)
        similarity_scores=F.cosine_similarity(text_embedding,gallery_embeddings)
        k=3
        top_scores,top_indices=torch.topk(similarity_scores,k=k)
        for i in range(k):
            score=top_scores[i][0].item()
            index=top_indices[i][0].item()
            filename=gallery_filenames[index]
            try:
                img=Image.open(filename)
                img.show()
            except Exception as e:
                print(f'error')
        

Enter a search query (or type 'q' to quit): dog
Enter a search query (or type 'q' to quit): food
Enter a search query (or type 'q' to quit): q


Exiting...
